# Nested Dissection

In [ ]:
import numpy as np
import scipy.sparse as spsp
import matplotlib.pyplot as plt
import time

def custom_lu_decomposition_iter(L, U, i, j, n, m):
    # Итерация для LU-разложения
    for k in range(j, j + m):
        a = U.getrow(k).getcol(k).toarray()[0][0]
        for row in range(max(i, k + 1), i + n):
            b = U.getrow(row).getcol(k).toarray()[0][0]
            if b != 0:
                L.append((row, k, b / (a + 1e-16)))
                U[row] = U.getrow(row) - b * (U.getrow(k) / (a + 1e-16))

def recursive_partitioning(P, L, U, i, n, separate_permutation):
    # Рекурсивное разбиение и применение LU-разложения
    if n < 10:  # Экстремальный случай
        custom_lu_decomposition_iter(L, U, i, i, n, n)
        return

    P_local, a, b, g = separate_permutation(U[i:(i+n), i:(i+n)])  # Получаем перестановку для блок-стрелочной формы
    P_i = np.arange(0, U.shape[0])  # Перестановка в i-й итерации
    P_i[i:(i+n)] = P_local + i
    U[:] = U[P_i, :][:, P_i]  # Применяем перестановку к U (в place)
    P[:] = P[P_i]

    if max(a, b, g) == n:  # Случай бесконечной рекурсии (когда следующее деление невозможно)
        custom_lu_decomposition_iter(L, U, i, i, n, n)
    else:
        recursive_partitioning(P, L, U, i, a, separate_permutation)  # (alpha, alpha)
        custom_lu_decomposition_iter(L, U, i + a + b, i, g, a)  # LU над частью (gamma,alpha)
        recursive_partitioning(P, L, U, i + a, b, separate_permutation)  # (beta, beta)
        custom_lu_decomposition_iter(L, U, i + a + b, i + a, g, b)  # LU над частью (gamma,beta)
        recursive_partitioning(P, L, U, i + a + b, g, separate_permutation)  # (gamma, gamma)

def lu_decomposition(A, separate_permutation, copy=True):
    # LU-разложение матрицы
    P = np.arange(0, A.shape[0])  # Перестановка по умолчанию
    L = [(i, i, 1) for i in range(0, A.shape[0])]  # Заполняем диагональ единицами

    if copy:
        U = A.copy()
    else:
        U = A

    # Изменения P, L, U вместе
    recursive_partitioning(P, L, U, 0, A.shape[0], separate_permutation)

    P = np.argsort(P)  # Применяем транспозицию

    # Создаем разреженную матрицу из списка COO
    row = [L[i][0] for i in range(len(L))]
    col = [L[i][1] for i in range(len(L))]
    data = [L[i][2] for i in range(len(L))]
    L = spsp.coo_matrix((data, (row, col)))

    return P, L, U

def generate_sparse_matrix(n, density=0.2):
    # Генерация разреженной матрицы размера n x n с указанной плотностью
    return spsp.rand(n, n, density=density, format='csr')

def find_separator_permutation(mat):
    # Находит собственный вектор Фидлера
    Laplacian = spsp.csgraph.laplacian(mat, dtype=np.float32)
    eigval, eigvec = spsp.linalg.eigsh(Laplacian, k=2, which="SM")
    colors = np.sign(eigvec[:, 1])

    # Получаем 3 множества, где gamma - разделитель
    if eigval[1] < 1e-8:  # Проверка на неподключенный граф
        alpha = spsp.csgraph.depth_first_order(Laplacian, 0, return_predecessors=False)
        beta = np.setdiff1d(np.arange(0, len(colors)), alpha)
        gamma = np.array([])
    else:
        gamma = set()
        for k in range(0, len(Laplacian.row)):
            i = Laplacian.row[k]
            j = Laplacian.col[k]
            if j > i and colors[i] != colors[j]:
                gamma.add(i)
                gamma.add(j)
        gamma = np.array(list(gamma))
        beta = np.setdiff1d(np.where(colors > 0), gamma)
        alpha = np.setdiff1d(np.where(colors < 0), gamma)

    # Создаем перестановку на основе множеств
    a, b, g = len(alpha), len(beta), len(gamma)
    start = 0
    P = np.zeros(len(colors), dtype=np.int32)
    if a > 0: P[alpha] = np.arange(start, start + a)
    start += a
    if b > 0: P[beta] = np.arange(start, start + b)
    start += b
    if g > 0: P[gamma] = np.arange(start, start + g)
    P = P.argsort()
    return P, a, b, g

# Пример использования
if __name__ == '__main__':
    # Генерация разреженной матрицы
    n = 10000
    density = 0.9
    A = generate_sparse_matrix(n, density)
    
    # Получение LU-разложения
    start_time = time.time()
    P, L, U = lu_decomposition(A, find_separator_permutation)
    end_time = time.time()
    print("Время разложения [с] =", end_time - start_time)
    
    # Проверка точности разложения
    LU = L.dot(U)
    norm_A = spsp.linalg.norm(A, ord='fro')
    norm_LU = spsp.linalg.norm(LU, ord='fro')
    norm_diff = spsp.linalg.norm(LU - A, ord='fro')

    print("Norm(A):", norm_A)
    print("Norm(LU):", norm_LU)
    print("Norm(P(LU)P - A):", norm_diff)

    # Визуализация разреженных матриц
    data = [A, LU, L, U]
    fig, axs = plt.subplots(1, len(data))
    for i in range(len(data)):
        axs[i].spy(data[i], aspect='equal', marker='.', markersize=2, precision=1e-5)
    plt.show()

In [143]:
%%time
P, L, U = lu_decomposition(A, find_separator_permutation)

CPU times: user 3.9 s, sys: 59.8 ms, total: 3.96 s
Wall time: 4.25 s


In [142]:
%%time
partial_pivoting_lu(A.todense())

CPU times: user 7.87 ms, sys: 2.83 ms, total: 10.7 ms
Wall time: 10.1 ms


(array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 1., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 array([[ 1.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.09878505,  1.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.9434324 , -0.17213849,  1.        , ...,  0.        ,
          0.        ,  0.        ],
        ...,
        [ 0.65144641,  0.01790458,  0.52061913, ...,  1.        ,
          0.        ,  0.        ],
        [ 0.44809754, -0.05302043,  0.47574243, ..., -0.84605615,
          1.        ,  0.        ],
        [ 0.        ,  0.88162689, -0.23215264, ..., -0.91704044,
          0.05211275,  1.        ]]),
 array([[ 9.93229556e-01,  1.86180951e-01,  9.70358373e-01, ...,
          1.16564073e-01,  2.67295380e-01,  5.93397063e-01]

In [132]:
%%time
lu(A.todense())

CPU times: user 756 µs, sys: 467 µs, total: 1.22 ms
Wall time: 3.39 ms


(array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 array([[ 1.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.37934975,  1.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.15775701,  0.85185965,  1.        , ...,  0.        ,
          0.        ,  0.        ],
        ...,
        [ 0.22400697,  0.4413075 ,  0.56373713, ...,  1.        ,
          0.        ,  0.        ],
        [ 0.07593386,  0.54473973,  0.43863608, ..., -0.14837557,
          1.        ,  0.        ],
        [ 0.0478587 ,  0.69108394,  0.14564925, ...,  0.46124642,
         -0.98556266,  1.        ]]),
 array([[ 0.98508833,  0.02581559,  0.67572781, ...,  0.63228184,
          0.11985304,  0.9560776 ],
        [ 0.        , 

In [141]:
def partial_pivoting_lu(A):
    n = len(A)
    P = np.eye(n)  # начальная перестановочная матрица
    L = np.eye(n)  # начальная нижняя треугольная матрица
    U = np.copy(A)  # начальная верхняя треугольная матрица

    for j in range(n - 1):
        # Шаг 1: Находим максимальный элемент в столбце под главной диагональю
        pivot_row = np.argmax(np.abs(U[j:, j])) + j

        # Меняем строки в U, L и P соответственно
        U[[j, pivot_row], j:] = U[[pivot_row, j], j:]
        L[[j, pivot_row], :j] = L[[pivot_row, j], :j]
        P[[j, pivot_row], :] = P[[pivot_row, j], :]

        # Шаг 2: Обновляем матрицы L и U
        L[j + 1:, j] = U[j + 1:, j] / U[j, j]
        U[j + 1:, j:] -= np.outer(L[j + 1:, j], U[j, j:])

    return P, L, U